In [3]:
from datasets import  concatenate_datasets, load_from_disk

org_dataset = load_from_disk('./data/train_dataset')
print(org_dataset)
full_ds = concatenate_datasets(
        [
            org_dataset["train"].flatten_indices(),
            org_dataset["validation"].flatten_indices(),
        ]
    )  # train dev 를 합친 4192 개 질문에 대해 모두 테스트
print("*" * 40, "query dataset", "*" * 40)
print(full_ds)

c:\Users\C4022\Desktop\study\coding\naver_boot\level2-mrc-nlp-07\level2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 240
    })
})
**************************************** query dataset ****************************************
Dataset({
    features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
    num_rows: 4192
})


In [5]:
correct_count = 0
incorrect_count = 0

for i in range(len(full_ds)):
    sample = full_ds[i]
    context = sample['context']
    answers = sample['answers']
    
    for answer in answers['text']:
        start = answers['answer_start'][0]
        extracted_answer = context[start:start + len(answer)]
        
        if extracted_answer == answer:
            correct_count += 1
        else:
            incorrect_count += 1

total = correct_count + incorrect_count
accuracy = (correct_count / total) * 100 if total > 0 else 0

print(f"총 샘플 수: {total}")
print(f"정확한 위치에 있는 답변 수: {correct_count}")
print(f"부정확한 위치에 있는 답변 수: {incorrect_count}")
print(f"정확도: {accuracy:.2f}%")

총 샘플 수: 4192
정확한 위치에 있는 답변 수: 4192
부정확한 위치에 있는 답변 수: 0
정확도: 100.00%


In [6]:
import textwrap
from IPython.display import clear_output

def review_dataset(dataset, batch_size=2):
    total_samples = len(dataset)
    current_index = 7

    # ANSI 색상 코드
    HIGHLIGHT_COLOR = '\033[92m'  # 녹색
    RESET_COLOR = '\033[0m'

    while current_index < total_samples:
        clear_output(wait=True)
        for i in range(current_index, min(current_index + batch_size, total_samples)):
            sample = dataset[i]
            start = sample['answers']['answer_start'][0]
            answer_text = sample['answers']['text'][0]
            end = start + len(answer_text)
            
            print(f"\n샘플 {i+1}/{total_samples}")
            print(f"제목: {sample['title']}")
            print(f"질문: {sample['question']}")
            print("내용:")
            
            context = sample['context']
            colored_context = (
                context[:start] +
                HIGHLIGHT_COLOR + context[start:end] + RESET_COLOR +
                context[end:]
            )
            
            print(textwrap.fill(colored_context, width=60))
            print(f"답변: {answer_text}")
            
            answer_context = context[start:start+20]
            print(f"답변 위치: {answer_context}")
            
            print("-" * 100)

        user_input = input("계속하려면 'g'를, 중지하려면 's'를 입력하세요: ").lower()
        if user_input == 's':
            print("검토를 중지합니다.")
            break
        elif user_input != 'g':
            print("잘못된 입력입니다")
            continue

        current_index += batch_size

    print("Fin")

# 데이터셋을 불러온 후 함수 호출
review_dataset(full_ds)


샘플 24/4192
제목: 셔틀맨
질문: 김건의 할아버지가 세운 학교는?
내용:
이영수\n\n키와 체중은 174cm에다 62kg이다. 혈액형은 B형이며, 셔틀맨의 남주인공이다. 고등학교
1학년 때 박일진에게 빵 심부름을 강요당하고 있었다.\n셔틀맨 인재를 발굴하기 위해 잠시 이영수의 학교에 온
김건을 만나 자신이 짝사랑하던 여학생 심연희와 함께 사립 김노인 고등학교로 간다. 김건의
말과 적성검사를 통해 자신이 셔틀맨에 소질이 있다는 사실을 알게 되고 독서와 신체적 훈련을 하면서
성정하고,방학 때 떠난 사막 훈련장에서 연희의 실수로 벗아났을 땐 길을 잃고 쓰러진 세 명의 아랍인들,택시
기름이 다 떨어진 택시기사와 택시,스핑크스를 기절한 연희,민석이를 부축한 채로 사막 훈련장에 도착하는
괴력까지 발휘한다.\n25살 때 SME의 직원이 된다.인기투표 순위는 4위.\n\n김건\n\n키와 체중은
186cm에 74kg. 또한 혈액형은 B형이며, 남학생치곤 긴 편인 머리를 반묶음으로 묶은 백발이며 훤칠한
미소년이다.\n어렸을 때부터 셔틀맨에 남다른 재능이 있었고,할아버지가 세운 학교로 갈 전학생을 모집하기 위해
가끔씩 다른 학교로 간다.입담이 매우 좋을 뿐 아니라 달리기도 잘하며,어렸을 때부터 훈련을 한 덕분에 몸매와
체력도 아주 좋으며 독서광이다.빵 심부름을 억지로 하는 영수와 달리 빵 심부름을 하는 것을 자랑스럽게
생각한다. 다만 독설을 잘하고 삿대질을 하는 버릇 때문에 중학교 시절 친구였던 찬희는 그런 건이를 싫어하게
되었고,영수 역시 그런 건이를 부담스러워 한다.\n사립 김노인 고등학교의 조교를 맡고 있다.25살 때
SME의 직원이 되었지만 여전히 조교를 맡고 있었다.인기투표 순위는 2위.\n\n심연희\n\n키와 체중은
164cm에 48kg. 혈액형은 O형이며, 눈매가 둥글둥글한 영수와 달리 눈매가 올라간 고양이 상이며
속눈썹이 짙은 편이다.\n청순한 이미지와 달리 특이한 성격 때문에 친구가 없다.예쁜 외모로 남학생들에게
인기가 많았지만 연희 본인은 매우 귀찮았으